<a href="https://colab.research.google.com/github/CJO100293/Crime-Data/blob/main/ETL/LA_Crime_Data_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing the Needed Dependencies
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', 400)
import os
import requests
import json
import csv
import geopandas as gpd
from pprint import pprint
from pathlib import Path
import ast
from google.colab import drive

In [2]:
# Mounting Google Drive Account
drive.mount('/content/drive')

Mounted at /content/drive


# Performing ETL on "Crime_Data_from_2020_to_Present.csv" Dataset

In [3]:
# Create reference to CSV file
url = "https://nextcloud.unknowntunnel.com/s/pQzfFatWdwNYNiR/download/Crime_Data_from_2020_to_Present.csv"

# Import the CSV into a pandas DataFrame
df_crime_data = pd.read_csv(url)

# Display dataframe
df_crime_data.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,10304468,01/08/2020 12:00:00 AM,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
1,190101086,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
2,200110444,04/14/2020 12:00:00 AM,02/13/2020 12:00:00 AM,1200,1,Central,155,2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,...,AA,Adult Arrest,845.0,NaN,NaN,NaN,200 E 6TH ST,NaN,34.0448,-118.2474
3,191501505,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,1730,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,IC,Invest Cont,745.0,998.0,NaN,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019
4,191921269,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",...,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468


In [4]:
# Get a brief summary of the df_crime_data DataFrame.
df_crime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 847725 entries, 0 to 847724
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DR_NO           847725 non-null  int64  
 1   Date Rptd       847725 non-null  object 
 2   DATE OCC        847725 non-null  object 
 3   TIME OCC        847725 non-null  int64  
 4   AREA            847725 non-null  int64  
 5   AREA NAME       847725 non-null  object 
 6   Rpt Dist No     847725 non-null  int64  
 7   Part 1-2        847725 non-null  int64  
 8   Crm Cd          847725 non-null  int64  
 9   Crm Cd Desc     847725 non-null  object 
 10  Mocodes         730083 non-null  object 
 11  Vict Age        847725 non-null  int64  
 12  Vict Sex        735816 non-null  object 
 13  Vict Descent    735808 non-null  object 
 14  Premis Cd       847714 non-null  float64
 15  Premis Desc     847209 non-null  object 
 16  Weapon Used Cd  295361 non-null  float64
 17  Weapon Des

In [5]:
# Dropping unneeded columns
df_crime_data = df_crime_data.drop(['DR_NO', 'TIME OCC', 'AREA', 'AREA NAME',
                                    'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Mocodes',
                                    'Premis Cd', 'Premis Desc', 'Weapon Used Cd',
                                    'Weapon Desc', 'Status', 'Status Desc',
                                    'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4',
                                    'LOCATION', 'Cross Street', 'DATE OCC'], axis=1)
# Display dataframe
df_crime_data.head()

,Date Rptd,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,LAT,LON
0,01/08/2020 12:00:00 AM,BATTERY - SIMPLE ASSAULT,36,F,B,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY - SIMPLE ASSAULT,25,M,H,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,X,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",31,X,X,34.2198,-118.4468


In [6]:
# Renaming the "Date Rptd" column to "Data Reported",
# "Crm Cd Desc" to "Crime",
# "Vict Age" to "Age",
# "Vict Sex" to "Gender"
# "Vict Descent" to "Race"
# "LAT" to "Latitude",
# and "LON" to "Longitude".
df_crime_data = df_crime_data.rename(columns={"Date Rptd":"Data Reported", "Crm Cd Desc":"Crime",
                                              "Vict Age":"Age", "Vict Sex":"Gender",
                                              "Vict Descent":"Race", "LAT":"Latitude", "LON":"Longitude"})
df_crime_data.head()

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY - SIMPLE ASSAULT,36,F,B,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY - SIMPLE ASSAULT,25,M,H,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,X,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",31,X,X,34.2198,-118.4468


In [7]:
# Replacing all instances of "ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER" and
# "ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT" and "OTHER ASSAULT" in the "Crime" column to "Assault"
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER': 'ASSAULT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT': 'ASSAULT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'OTHER ASSAULT': 'ASSAULT'})
df_crime_data

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY - SIMPLE ASSAULT,36,F,B,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY - SIMPLE ASSAULT,25,M,H,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,X,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",31,X,X,34.2198,-118.4468
...,...,...,...,...,...,...,...
847720,03/22/2023 12:00:00 AM,ASSAULT,25,F,H,34.2790,-118.4116
847721,04/12/2023 12:00:00 AM,ASSAULT,29,M,B,33.9841,-118.2915
847722,07/02/2023 12:00:00 AM,PICKPOCKET,24,F,H,34.0467,-118.2485
847723,03/05/2023 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),53,F,H,34.1951,-118.4487


In [8]:
# Replacing all instances of "BATTERY - SIMPLE ASSAULT", "BATTERY ON A FIREFIGHTER", "BATTERY POLICE (SIMPLE)" and
# "BATTERY WITH SEXUAL CONTACT" in the "Crime" column to "Battery"
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'BATTERY - SIMPLE ASSAULT': 'BATTERY'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'BATTERY ON A FIREFIGHTER': 'BATTERY'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'BATTERY POLICE (SIMPLE)': 'BATTERY'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'BATTERY WITH SEXUAL CONTACT': 'BATTERY'})
df_crime_data

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY,36,F,B,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY,25,M,H,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,X,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",31,X,X,34.2198,-118.4468
...,...,...,...,...,...,...,...
847720,03/22/2023 12:00:00 AM,ASSAULT,25,F,H,34.2790,-118.4116
847721,04/12/2023 12:00:00 AM,ASSAULT,29,M,B,33.9841,-118.2915
847722,07/02/2023 12:00:00 AM,PICKPOCKET,24,F,H,34.0467,-118.2485
847723,03/05/2023 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),53,F,H,34.1951,-118.4487


In [9]:
# Replacing all instances of "BURGLARY FROM VEHICLE", "BURGLARY FROM VEHICLE, ATTEMPTED", and
# "BURGLARY, ATTEMPTED" in the "Crime" column to "BURGLARY"
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'BURGLARY FROM VEHICLE': 'BURGLARY'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'BURGLARY FROM VEHICLE': 'BURGLARY'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'BURGLARY, ATTEMPTED': 'BURGLARY'})
df_crime_data

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY,36,F,B,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY,25,M,H,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,X,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",31,X,X,34.2198,-118.4468
...,...,...,...,...,...,...,...
847720,03/22/2023 12:00:00 AM,ASSAULT,25,F,H,34.2790,-118.4116
847721,04/12/2023 12:00:00 AM,ASSAULT,29,M,B,33.9841,-118.2915
847722,07/02/2023 12:00:00 AM,PICKPOCKET,24,F,H,34.0467,-118.2485
847723,03/05/2023 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),53,F,H,34.1951,-118.4487


In [10]:
# Replacing all instances of "CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT", and
# "CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT" in the "Crime" column to "CHILD ABUSE"
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT': 'CHILD ABUSE'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT': 'CHILD ABUSE'})
df_crime_data

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY,36,F,B,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY,25,M,H,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,X,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",31,X,X,34.2198,-118.4468
...,...,...,...,...,...,...,...
847720,03/22/2023 12:00:00 AM,ASSAULT,25,F,H,34.2790,-118.4116
847721,04/12/2023 12:00:00 AM,ASSAULT,29,M,B,33.9841,-118.2915
847722,07/02/2023 12:00:00 AM,PICKPOCKET,24,F,H,34.0467,-118.2485
847723,03/05/2023 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),53,F,H,34.1951,-118.4487


In [11]:
# Replacing all instances of "CREDIT CARDS, FRAUD USE ($950 & UNDER", and
# "CREDIT CARDS, FRAUD USE ($950.01 & OVER)" in the "Crime" column to "CREDIT CARD FRAUD"
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'CREDIT CARDS, FRAUD USE ($950 & UNDER': 'CREDIT CARD FRAUD'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'CREDIT CARDS, FRAUD USE ($950.01 & OVER)': 'CREDIT CARD FRAUD'})
df_crime_data

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY,36,F,B,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY,25,M,H,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,X,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",31,X,X,34.2198,-118.4468
...,...,...,...,...,...,...,...
847720,03/22/2023 12:00:00 AM,ASSAULT,25,F,H,34.2790,-118.4116
847721,04/12/2023 12:00:00 AM,ASSAULT,29,M,B,33.9841,-118.2915
847722,07/02/2023 12:00:00 AM,PICKPOCKET,24,F,H,34.0467,-118.2485
847723,03/05/2023 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),53,F,H,34.1951,-118.4487


In [12]:
# Replacing all instances of "BUNCO, PETTY THEFT",
#"BUNCO, GRAND THEFT",
#"DEFRAUDING INNKEEPER/THEFT OF SERVICES, $950 & UNDER",
#"DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $950.01",
#"DISHONEST EMPLOYEE - GRAND THEFT",
#"DISHONEST EMPLOYEE - PETTY THEFT",
#"DISHONEST EMPLOYEE ATTEMPTED THEFT",
#"EMBEZZLEMENT, GRAND THEFT ($950.01 & OVER)",
#"EMBEZZLEMENT, PETTY THEFT ($950 & UNDER)",
#"GRAND THEFT / INSURANCE FRAUD",
#"GRAND THEFT / AUTO REPAIR",
#"PETTY THEFT / AUTO REPAIR",
#"SHOPLIFTING - PETTY THEFT ($950 & UNDER)",
#"SHOPLIFTING-GRAND THEFT ($950.01 & OVER)",
#"THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)",
#"THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND OVER)",
#"THEFT FROM MOTOR VEHICLE - ATTEMPT",
#"THEFT FROM PERSON - ATTEMPT",
#"THEFT PLAIN - PETTY ($950 & UNDER)",
#"THEFT PLAIN - ATTEMPT",
#"THEFT, COIN MACHINE - PETTY ($950 & UNDER)",
#"THEFT, COIN MACHINE - GRAND ($950.01 & OVER)",
#"THEFT, COIN MACHINE - ATTEMPT",
#"THEFT, PERSON",
#"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD", and
#"TILL TAP - GRAND THEFT ($950.01 & OVER)" in the "Crime" column to "THEFT"
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'BUNCO, PETTY THEFT': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'BUNCO, GRAND THEFT': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'DEFRAUDING INNKEEPER/THEFT OF SERVICES, $950 & UNDER': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $950.01': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'DISHONEST EMPLOYEE - GRAND THEFT': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'DISHONEST EMPLOYEE - PETTY THEFT': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'DISHONEST EMPLOYEE ATTEMPTED THEFT': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'EMBEZZLEMENT, GRAND THEFT ($950.01 & OVER)': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'EMBEZZLEMENT, PETTY THEFT ($950 & UNDER)': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'GRAND THEFT / INSURANCE FRAUD': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'GRAND THEFT / AUTO REPAIR': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'PETTY THEFT / AUTO REPAIR': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'SHOPLIFTING - PETTY THEFT ($950 & UNDER)': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'SHOPLIFTING-GRAND THEFT ($950.01 & OVER)': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND OVER)': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'THEFT FROM MOTOR VEHICLE - ATTEMPT': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'THEFT FROM PERSON - ATTEMPT': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'THEFT PLAIN - PETTY ($950 & UNDER)': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'THEFT PLAIN - ATTEMPT': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'THEFT, COIN MACHINE - PETTY ($950 & UNDER)': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'THEFT, COIN MACHINE - GRAND ($950.01 & OVER)': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'THEFT, COIN MACHINE - ATTEMPT': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'THEFT, PERSON': 'BUNCO'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD': 'THEFT'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'TILL TAP - GRAND THEFT ($950.01 & OVER)': 'THEFT'})
df_crime_data

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY,36,F,B,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY,25,M,H,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,X,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",31,X,X,34.2198,-118.4468
...,...,...,...,...,...,...,...
847720,03/22/2023 12:00:00 AM,ASSAULT,25,F,H,34.2790,-118.4116
847721,04/12/2023 12:00:00 AM,ASSAULT,29,M,B,33.9841,-118.2915
847722,07/02/2023 12:00:00 AM,PICKPOCKET,24,F,H,34.0467,-118.2485
847723,03/05/2023 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),53,F,H,34.1951,-118.4487


In [13]:
# Replacing all instances of "THEFT OF IDENTITY" in the "Crime" column to "IDENTITY THEFT"
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'THEFT OF IDENTITY': 'IDENTITY THEFT'})
df_crime_data

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY,36,F,B,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY,25,M,H,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,X,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",31,X,X,34.2198,-118.4468
...,...,...,...,...,...,...,...
847720,03/22/2023 12:00:00 AM,ASSAULT,25,F,H,34.2790,-118.4116
847721,04/12/2023 12:00:00 AM,ASSAULT,29,M,B,33.9841,-118.2915
847722,07/02/2023 12:00:00 AM,PICKPOCKET,24,F,H,34.0467,-118.2485
847723,03/05/2023 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),53,F,H,34.1951,-118.4487


In [14]:
# Replacing all instances of "HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE" and "HUMAN TRAFFICKING - COMMERCIAL SEX ACTS"
# in the "Crime" column to "HUMAN TRAFFICKING"
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE': 'HUMAN TRAFFICKING'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'HUMAN TRAFFICKING - COMMERCIAL SEX ACTS': 'HUMAN TRAFFICKING'})
df_crime_data

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY,36,F,B,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY,25,M,H,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,X,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",31,X,X,34.2198,-118.4468
...,...,...,...,...,...,...,...
847720,03/22/2023 12:00:00 AM,ASSAULT,25,F,H,34.2790,-118.4116
847721,04/12/2023 12:00:00 AM,ASSAULT,29,M,B,33.9841,-118.2915
847722,07/02/2023 12:00:00 AM,PICKPOCKET,24,F,H,34.0467,-118.2485
847723,03/05/2023 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),53,F,H,34.1951,-118.4487


In [15]:
# Replacing all instances of "INTIMATE PARTNER - SIMPLE ASSAULT" and "INTIMATE PARTNER - AGGRAVATED ASSAULT"
#in the "Crime" column to "DOMESTIC VIOLENCE"
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'INTIMATE PARTNER - SIMPLE ASSAULT': 'DOMESTIC VIOLENCE'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'INTIMATE PARTNER - AGGRAVATED ASSAULT': 'DOMESTIC VIOLENCE'})
df_crime_data

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY,36,F,B,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY,25,M,H,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,X,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",31,X,X,34.2198,-118.4468
...,...,...,...,...,...,...,...
847720,03/22/2023 12:00:00 AM,ASSAULT,25,F,H,34.2790,-118.4116
847721,04/12/2023 12:00:00 AM,ASSAULT,29,M,B,33.9841,-118.2915
847722,07/02/2023 12:00:00 AM,PICKPOCKET,24,F,H,34.0467,-118.2485
847723,03/05/2023 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),53,F,H,34.1951,-118.4487


In [16]:
# Replacing all instances of "KIDNAPPING - GRAND ATTEMPT" in the "Crime" column to "KIDNAPPING"
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'KIDNAPPING - GRAND ATTEMPT': 'KIDNAPPING'})
df_crime_data

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY,36,F,B,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY,25,M,H,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,X,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",31,X,X,34.2198,-118.4468
...,...,...,...,...,...,...,...
847720,03/22/2023 12:00:00 AM,ASSAULT,25,F,H,34.2790,-118.4116
847721,04/12/2023 12:00:00 AM,ASSAULT,29,M,B,33.9841,-118.2915
847722,07/02/2023 12:00:00 AM,PICKPOCKET,24,F,H,34.0467,-118.2485
847723,03/05/2023 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),53,F,H,34.1951,-118.4487


In [17]:
# Replacing all instances of "LEWD/LASCIVIOUS ACTS WITH CHILD" in the "Crime" column to "LEWD CONDUCT"
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'LEWD/LASCIVIOUS ACTS WITH CHILD': 'LEWD CONDUCT'})
df_crime_data

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY,36,F,B,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY,25,M,H,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,X,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",31,X,X,34.2198,-118.4468
...,...,...,...,...,...,...,...
847720,03/22/2023 12:00:00 AM,ASSAULT,25,F,H,34.2790,-118.4116
847721,04/12/2023 12:00:00 AM,ASSAULT,29,M,B,33.9841,-118.2915
847722,07/02/2023 12:00:00 AM,PICKPOCKET,24,F,H,34.0467,-118.2485
847723,03/05/2023 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),53,F,H,34.1951,-118.4487


In [18]:
# Replacing all instances of "LYNCHING - ATTEMPTED" in the "Crime" column to "LYNCHING"
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'LYNCHING - ATTEMPTED': 'LYNCHING'})
df_crime_data

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY,36,F,B,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY,25,M,H,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,X,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",31,X,X,34.2198,-118.4468
...,...,...,...,...,...,...,...
847720,03/22/2023 12:00:00 AM,ASSAULT,25,F,H,34.2790,-118.4116
847721,04/12/2023 12:00:00 AM,ASSAULT,29,M,B,33.9841,-118.2915
847722,07/02/2023 12:00:00 AM,PICKPOCKET,24,F,H,34.0467,-118.2485
847723,03/05/2023 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),53,F,H,34.1951,-118.4487


In [19]:
# Replacing all instances of "PICKPOCKET, ATTEMPT" in the "Crime" column to "PICKPOCKET"
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'PICKPOCKET, ATTEMPT': 'PICKPOCKET'})
df_crime_data

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY,36,F,B,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY,25,M,H,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,X,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",31,X,X,34.2198,-118.4468
...,...,...,...,...,...,...,...
847720,03/22/2023 12:00:00 AM,ASSAULT,25,F,H,34.2790,-118.4116
847721,04/12/2023 12:00:00 AM,ASSAULT,29,M,B,33.9841,-118.2915
847722,07/02/2023 12:00:00 AM,PICKPOCKET,24,F,H,34.0467,-118.2485
847723,03/05/2023 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),53,F,H,34.1951,-118.4487


In [20]:
# Replacing all instances of "PURSE SNATCHING - ATTEMPT" in the "Crime" column to "PURSE SNATCHING"
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'PURSE SNATCHING - ATTEMPT': 'PURSE SNATCHING'})
df_crime_data

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY,36,F,B,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY,25,M,H,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,X,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",31,X,X,34.2198,-118.4468
...,...,...,...,...,...,...,...
847720,03/22/2023 12:00:00 AM,ASSAULT,25,F,H,34.2790,-118.4116
847721,04/12/2023 12:00:00 AM,ASSAULT,29,M,B,33.9841,-118.2915
847722,07/02/2023 12:00:00 AM,PICKPOCKET,24,F,H,34.0467,-118.2485
847723,03/05/2023 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),53,F,H,34.1951,-118.4487


In [21]:
# Replacing all instances of "RAPE, FORCIBLE" and "RAPE, ATTEMPTED" in the "Crime" column to "RAPE"
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'RAPE, FORCIBLE': 'RAPE'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'RAPE, ATTEMPTED': 'RAPE'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'SEX,UNLAWFUL(INC MUTUAL CONSENT, PENETRATION W/ FRGN OBJ':
                                                         'RAPE'})
df_crime_data

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY,36,F,B,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY,25,M,H,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,X,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",31,X,X,34.2198,-118.4468
...,...,...,...,...,...,...,...
847720,03/22/2023 12:00:00 AM,ASSAULT,25,F,H,34.2790,-118.4116
847721,04/12/2023 12:00:00 AM,ASSAULT,29,M,B,33.9841,-118.2915
847722,07/02/2023 12:00:00 AM,PICKPOCKET,24,F,H,34.0467,-118.2485
847723,03/05/2023 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),53,F,H,34.1951,-118.4487


In [22]:
# Replacing all instances of "SHOTS FIRED AT MOVING VEHICLE, TRAIN OR AIRCRAFT" and
#"SHOTS FIRED AT INHABITED DWELLING" in the "Crime" column to "SHOTS FIRED"
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'SHOTS FIRED AT MOVING VEHICLE, TRAIN OR AIRCRAFT':
                                                         'SHOTS FIRED'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'SHOTS FIRED AT INHABITED DWELLING':
                                                         'SHOTS FIRED'})
df_crime_data

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY,36,F,B,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY,25,M,H,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,X,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",31,X,X,34.2198,-118.4468
...,...,...,...,...,...,...,...
847720,03/22/2023 12:00:00 AM,ASSAULT,25,F,H,34.2790,-118.4116
847721,04/12/2023 12:00:00 AM,ASSAULT,29,M,B,33.9841,-118.2915
847722,07/02/2023 12:00:00 AM,PICKPOCKET,24,F,H,34.0467,-118.2485
847723,03/05/2023 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),53,F,H,34.1951,-118.4487


In [23]:
# Replacing all instances of "VANDALISM - MISDEAMEANOR ($399 OR UNDER)" and
#"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)" in the "Crime" column to "VANDALISM"
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'VANDALISM - MISDEAMEANOR ($399 OR UNDER)': 'VANDALISM'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)':
                                                         'VANDALISM'})
df_crime_data

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY,36,F,B,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY,25,M,H,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,X,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM,76,F,W,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,VANDALISM,31,X,X,34.2198,-118.4468
...,...,...,...,...,...,...,...
847720,03/22/2023 12:00:00 AM,ASSAULT,25,F,H,34.2790,-118.4116
847721,04/12/2023 12:00:00 AM,ASSAULT,29,M,B,33.9841,-118.2915
847722,07/02/2023 12:00:00 AM,PICKPOCKET,24,F,H,34.0467,-118.2485
847723,03/05/2023 12:00:00 AM,VANDALISM,53,F,H,34.1951,-118.4487


In [24]:
# Replacing all instances of "VEHICLE - ATTEMPT STOLEN", "VEHICLE - STOLEN" and
# "VEHICLE, STOLEN - OTHER (MOTORIZED SCOOTERS, BIKES, ETC)" in the "Crime" column to "STOLEN VEHICLE"
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'VEHICLE - ATTEMPT STOLEN': 'STOLEN VEHICLE'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'VEHICLE - STOLEN': 'STOLEN VEHICLE'})
df_crime_data['Crime'] = df_crime_data['Crime'].replace({'VEHICLE, STOLEN - OTHER (MOTORIZED SCOOTERS, BIKES, ETC)':
                                                         'STOLEN VEHICLE'})
df_crime_data

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY,36,F,B,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY,25,M,H,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,X,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM,76,F,W,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,VANDALISM,31,X,X,34.2198,-118.4468
...,...,...,...,...,...,...,...
847720,03/22/2023 12:00:00 AM,ASSAULT,25,F,H,34.2790,-118.4116
847721,04/12/2023 12:00:00 AM,ASSAULT,29,M,B,33.9841,-118.2915
847722,07/02/2023 12:00:00 AM,PICKPOCKET,24,F,H,34.0467,-118.2485
847723,03/05/2023 12:00:00 AM,VANDALISM,53,F,H,34.1951,-118.4487


In [25]:
# renaming "M" to "Male" and "F" to "Female" and "X" to "Unknown"  in the "Gender" column
df_crime_data['Gender'] = df_crime_data['Gender'].replace({'M': 'Male'})
df_crime_data['Gender'] = df_crime_data['Gender'].replace({'F': 'Female'})
df_crime_data['Gender'] = df_crime_data['Gender'].replace({'X': 'Unknown'})
df_crime_data

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY,36,Female,B,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY,25,Male,H,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,Unknown,X,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM,76,Female,W,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,VANDALISM,31,Unknown,X,34.2198,-118.4468
...,...,...,...,...,...,...,...
847720,03/22/2023 12:00:00 AM,ASSAULT,25,Female,H,34.2790,-118.4116
847721,04/12/2023 12:00:00 AM,ASSAULT,29,Male,B,33.9841,-118.2915
847722,07/02/2023 12:00:00 AM,PICKPOCKET,24,Female,H,34.0467,-118.2485
847723,03/05/2023 12:00:00 AM,VANDALISM,53,Female,H,34.1951,-118.4487


In [26]:
# renaming "A" to "Other Asian", "B" to "Black", "C" to "Chinese", "D" to "Cambodian", "F" to "Filipino",
# "G" to "Guamanian", "H" to "Hispanic/Latin/Mexican", "I" to "American Indian/Alaskan Native",
# "J" to "Japanese", "K" to "Korean", "L" to "Laotian", "O" to "Other", "P" to "Pacific Islander",
# "S" to "Samoan", "U" to "Hawaiian", "V" to "Vietnamese", "W" to "White", "X" to "Unknown",
# "Z" to "Asian Indian" in the "Race" column
df_crime_data['Race'] = df_crime_data['Race'].replace({'A': 'Other Asian'})
df_crime_data['Race'] = df_crime_data['Race'].replace({'B': 'Black'})
df_crime_data['Race'] = df_crime_data['Race'].replace({'C': 'Chinese'})
df_crime_data['Race'] = df_crime_data['Race'].replace({'D': 'Cambodian'})
df_crime_data['Race'] = df_crime_data['Race'].replace({'F': 'Filipino'})
df_crime_data['Race'] = df_crime_data['Race'].replace({'G': 'Guamanian'})
df_crime_data['Race'] = df_crime_data['Race'].replace({'H': 'Hispanic/Latin/Mexican'})
df_crime_data['Race'] = df_crime_data['Race'].replace({'I': 'American Indian/Alaskan Native'})
df_crime_data['Race'] = df_crime_data['Race'].replace({'J': 'Japanese'})
df_crime_data['Race'] = df_crime_data['Race'].replace({'K': 'Korean'})
df_crime_data['Race'] = df_crime_data['Race'].replace({'L': 'Laotian'})
df_crime_data['Race'] = df_crime_data['Race'].replace({'O': 'Other'})
df_crime_data['Race'] = df_crime_data['Race'].replace({'P': 'Pacific Islander'})
df_crime_data['Race'] = df_crime_data['Race'].replace({'S': 'Samoan'})
df_crime_data['Race'] = df_crime_data['Race'].replace({'U': 'Hawaiian'})
df_crime_data['Race'] = df_crime_data['Race'].replace({'V': 'Vietnamese'})
df_crime_data['Race'] = df_crime_data['Race'].replace({'W': 'White'})
df_crime_data['Race'] = df_crime_data['Race'].replace({'X': 'Unknown'})
df_crime_data['Race'] = df_crime_data['Race'].replace({'Z': 'Asian Indian'})
df_crime_data

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY,36,Female,Black,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY,25,Male,Hispanic/Latin/Mexican,34.0459,-118.2545
2,04/14/2020 12:00:00 AM,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,Unknown,Unknown,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,VANDALISM,76,Female,White,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,VANDALISM,31,Unknown,Unknown,34.2198,-118.4468
...,...,...,...,...,...,...,...
847720,03/22/2023 12:00:00 AM,ASSAULT,25,Female,Hispanic/Latin/Mexican,34.2790,-118.4116
847721,04/12/2023 12:00:00 AM,ASSAULT,29,Male,Black,33.9841,-118.2915
847722,07/02/2023 12:00:00 AM,PICKPOCKET,24,Female,Hispanic/Latin/Mexican,34.0467,-118.2485
847723,03/05/2023 12:00:00 AM,VANDALISM,53,Female,Hispanic/Latin/Mexican,34.1951,-118.4487


In [27]:
# Dropping all rows that contain "-1", "-2", "-3" or "0"
df_crime_data = df_crime_data[df_crime_data.Age != -1]
df_crime_data = df_crime_data[df_crime_data.Age != -2]
df_crime_data = df_crime_data[df_crime_data.Age != -3]
df_crime_data = df_crime_data[df_crime_data.Age != 0]
df_crime_data

,Data Reported,Crime,Age,Gender,Race,Latitude,Longitude
0,01/08/2020 12:00:00 AM,BATTERY,36,Female,Black,34.0141,-118.2978
1,01/02/2020 12:00:00 AM,BATTERY,25,Male,Hispanic/Latin/Mexican,34.0459,-118.2545
3,01/01/2020 12:00:00 AM,VANDALISM,76,Female,White,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,VANDALISM,31,Unknown,Unknown,34.2198,-118.4468
5,01/02/2020 12:00:00 AM,RAPE,25,Female,Hispanic/Latin/Mexican,34.0452,-118.2534
...,...,...,...,...,...,...,...
847720,03/22/2023 12:00:00 AM,ASSAULT,25,Female,Hispanic/Latin/Mexican,34.2790,-118.4116
847721,04/12/2023 12:00:00 AM,ASSAULT,29,Male,Black,33.9841,-118.2915
847722,07/02/2023 12:00:00 AM,PICKPOCKET,24,Female,Hispanic/Latin/Mexican,34.0467,-118.2485
847723,03/05/2023 12:00:00 AM,VANDALISM,53,Female,Hispanic/Latin/Mexican,34.1951,-118.4487


### Exporting Transformed Dataframe to CSV

In [28]:
# Export the newly transformed dataframe into a csv
df_crime_data.to_csv("/content/drive/My Drive/Colab-Notebooks/Crime Data/OUTPUT_DATA/ETL_Crime_Data.csv", index=False)

In [29]:
# Un-Mounting Google Drive Account
drive.flush_and_unmount()